**AGENT INTRODUCTION**

In [1]:
GROQ_API_KEY="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"


In [2]:
!pip install openai langchain langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00


In [3]:
!pip install -U langchain langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.6 MB/s eta 0:00:00


**Customer FeedBack Agent**

In [5]:
import os

# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GROQ_API_KEY"]   = GROQ_API_KEY

In [7]:
from dotenv import load_dotenv
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain_groq import ChatGroq

# Set up OpenAI LLM
#llm = ChatOpenAI(temperature=0.7, model="gpt-4")
llm = ChatGroq(model="llama3-70b-8192", temperature=0.7)

VERBOSE = False

# Custom function to analyze feedback
def comment_on_feedback(feedback: str) -> str:
    return f"Here's a comment on the feedback: \"{feedback}\""

# Wrap it in a LangChain Tool
feedback_tool = Tool(
    name="FeedbackCommenter",
    func=comment_on_feedback,
    description="Use this tool to analyze and comment on customer feedback."
)

# Initialize Agent with the tool
agent = initialize_agent(
    tools=[feedback_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=VERBOSE
)

# Provide a feedback sample
customer_feedback = "The food are disgusting"

# Run agent
response = agent.run(f"Please comment professionally on this customer feedback: '{customer_feedback}'")
print(response)


We apologize for the poor experience with our food. We take all feedback seriously and would like to make it right. Could you please provide more details about your visit so we can investigate and improve our offerings?


**LLAMA INDEX AGENTS**

In [1]:
!pip install nest_asyncio
!pip install llama-index
!pip install llama-index-llms-groq


In [4]:
import nest_asyncio
import asyncio
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.groq import Groq
from llama_index.core.memory import ChatMemoryBuffer
import re

# Fix event loop issue in Google Colab
nest_asyncio.apply()

# Initialize LLM
llm = Groq(model="llama3-70b-8192", api_key="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")

# Define mathematical operations with result tracking
last_result = None  # Stores the last numeric result

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and store the result."""
    global last_result
    last_result = a * b
    return last_result

def add(a: float, b: float) -> float:
    """Add two numbers and store the result."""
    global last_result
    last_result = a + b
    return last_result

def subtract(a: float, b: float) -> float:
    """Subtract two numbers and store the result."""
    global last_result
    last_result = a - b
    return last_result

def divide(a: float, b: float) -> float:
    """Divide two numbers, handling division by zero, and store the result."""
    global last_result
    if b == 0:
        return "Cannot divide by zero"
    last_result = a / b
    return last_result

# Initialize memory with a larger token limit
memory = ChatMemoryBuffer.from_defaults(token_limit=2048)

# Create an enhanced agent workflow with memory
agent = FunctionAgent(
    name="MathAgent",
    description="A math assistant that performs calculations and remembers past results.",
    tools=[multiply, add, subtract, divide],  # Adding more tools
    llm=llm,
    memory=memory,  # Enabling memory
    system_prompt=(
        "You are an intelligent math assistant. You remember past calculations and use them in follow-up questions. "
        "If the user refers to 'that' or 'previous answer', use the last computed numerical result."
    ),
)

# Function to extract the last numerical response from memory
def extract_last_number(response):
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", str(response))  # Extract numbers
    return float(numbers[-1]) if numbers else None  # Return last detected number

# Run async commands directly in a cell
async def test_agent():
    global last_result

    response1 = await agent.run(user_msg="What is 20 + 5?")
    last_result = extract_last_number(response1)  # Store last number explicitly
    print("Response 1:", response1)

    response2 = await agent.run(user_msg=f"Multiply {last_result} by 3")  # Explicit reference
    last_result = extract_last_number(response2)
    print("Response 2:", response2)

    response3 = await agent.run(user_msg=f"Now subtract 4 from {last_result}")  # Clear phrasing
    last_result = extract_last_number(response3)
    print("Response 3:", response3)

# Run the async function
await test_agent()


Response 1: The answer is 25.
Response 2: The result is 75.
Response 3: The result is 71.
